각 기능이 어떻게 동작하는지!
부제 : few-shot으로 원하는 답변을 얻을 수 있을까? (여러가지 분석이 가능할까? 
생각 :
우리가 룰베이스드 모든 단계를 작성하는 것은 절대적으로 불가능하다고 판단.
각 단계를 LLM으로 대체하기로 결정하였다. (학습을 위해)
필요한 기능 목록은 다음과 같다. [의도 분석기, 화행 분류, 질문 정제기, 쿼리 병합기]

먼저 각 기능의 I/O가 어떠한 형식인지 알아보고, few shot을 줬을 때 (prompting) 우리가 원하는 결과를 뽑을 수 있을지 확인해보자.

few shot 으로 원하는 답을 도출할 수 없다면, 어떠한 방법을 사용할 수 있을지도 생각해보자.

차후 시간이 된다면 latency (성능 지표는 객관적으로 평가하기 어렵기도 하고, 현 LLM 모델이 '뇌'로써 판단은 충분히 잘 한다고 판단, 측정하지 않기로 함)를 측정하고, 이를 개선할 방안이 있을 지 생각해보자. 

1. 질의 분석 : 사용자의 복합 질문을 독립적인 단순 질문으로 분해(parsing)하는 단계
예: 강남구 전기차 충전소 이름과 주소 알려줘 
-> 강남구 전기차 충전소 이름 알려줘
-> 강남구 전기차 충전소 주소 알려줘


In [1]:
# 필요 패키지 import 및 API 키 설정 로드
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.callbacks import StreamingStdOutCallbackHandler
load_dotenv()

True

In [2]:
# OpenAI 모델 객체 생성
llm = ChatOpenAI(
    model="gpt-4o-mini", # 경량화 시 3.5 turbo 등 선택지 존재. 
    temperature=1,# temperature : 모델의 창의성을 조절하는 하이퍼파라미터 
    # 낮으면 -> 같은 질문엔 같은 답변 / 높으면 -> 같은 질문이더라도 다양한 답변
    # 매 토큰마다 다음에 올 토큰의 확률을 게산하는데, temp가 높을 수록 낮은 확률의 단어를 고를 가능성이 증가.
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()], # 그럼 그 핸들러로는 뭘 쓸지
    api_key=os.getenv("OPENAI_API_KEY") # API 키 설정
)

# 질의 분석
- 사용자의 복합 질문을 독립적인 단순 질문으로 분해(parsing)하는 단계 <br>
예: 강남구 전기차 충전소 이름과 주소 알려줘 <br>
\-> 강남구 전기차 충전소 이름 알려줘 <br>
\-> 강남구 전기차 충전소 주소 알려줘

In [3]:
# few-shot 예제 생성 based on langchain
# chat gpt api 기반과 비교해보기. 성능 상 무엇이 더 강한지.

from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate


In [4]:

# 1. 시스템 프롬프트 정의
system_prompt =  """
당신은 '서울 열린데이터 광장'의 RAG 프로세스 중 일부입니다.
주어진 정보 외 창작은 금지합니다.
당신의 임무는 질문에 답을 하는 것이 아니라, 질문을 보고 질문을 나누는 것입니다.(질문 재가공하여 창작하지마시오.)
질문이 여러개의 질문으로 합쳐진 형태이면, 의도별로 질문을 나눠주십시오.
나눠진 질문은 서로 독립적입니다. 나눈 질문을 "위를 데이터로 보여줘" 등 연관성을 갖게 나누지 말고 각각 질문으로써 의미를 갖게 하시오.
질문이 단일 의도를 지녔다면, 질문을 나누지 마십시오.
질문에 "?" 기호가 있다면, "?" 기호를 그대로 적용하십시오.

주의사항:
단어로만 이루어진 질문은 단어를 분리만 합니다.
주어진 질문만을 사용하여 질문을 분리하고, 추가 정보를 이용하지 않습니다.
질문에 지시문이 함께 있는 경우 제외하고 결과에 반영하지 마시오.
무엇에 대해 알려달라는 질문이면, 질문을 나누지 마십시오.
질문의 의도가 불분명한 경우, 질문을 새롭게 창조하지 않고, 주어진 방식대로만 분리합니다.
질문 재가공하여 창작하는 것을 금지합니다.
주어진 텍스트 외의 추가사항 넣지마.
"~비교분석해줘"는 "~비교해줘", "~분석해줘"로 나누지 않고 "~비교분석해줘"로 답변합니다.
예를들어 "자치구별 전세 보증금 비교분석해줘"는 "● 자치구별 전세 보증금 비교분석해줘"로 답변합니다.
"""

In [5]:
# 2. few-shot 예제 데이터 정의

# --- 2. Few-shot 예제 데이터 정의 ---
examples = [
    {
        "input": "횡단보도 교통사고",
        "output": "● 횡단보도\n● 교통사고"
    },
    {
        "input": "강남구에 있는 급속 충전이 가능한 전기차 충전소의 이름과 주소와 주차료 부과 여부에 대해 알려줘",
        "output": "● 강남구에 있는 급속 충전이 가능한 전기차 충전소의 이름에 대해 알려줘\n● 강남구에 있는 급속 충전이 가능한 전기차 충전소의 주소에 대해 알려줘\n● 강남구에 있는 급속 충전이 가능한 전기차 충전소의 주차료 부과 여부에 대해 알려줘"
    },
    {
        "input": "성동구에 위치한 제설함의 개수는 총 몇 개야?",
        "output": "● 성동구에 위치한 제설함의 개수는 총 몇 개야?"
    },
    {
        "input": "강남구에 있는 헬스장에 대해 알려줘",
        "output": "● 강남구에 있는 헬스장에 대해 알려줘"
    },
    {
        "input": "방범용 CCTV가 많을수록 범죄 발생이 적은지 구별로 비교해줘",
        "output": "● 구별로 방범용 CCTV 개수 비교해줘\n● 구별로 범죄 발생 수 비교해줘"
    },
    {
        "input": "자치구별 공원 수 비교분석해줘",
        "output": "● 자치구별 공원 수 비교분석해줘"
    }
]

In [6]:
# 3. 플릿 정의
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# 4. few-shot 프롬프트 템플릿 생성
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

In [7]:
# 5. 최종 프롬프트 템플릿 조립

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

In [9]:
# 6. 체인 구성
# 체인이란? 여러 구성 요소(LLM, 프롬프트 템플릿, 파서)를 연결하여 특정 작업을 수행하는 일련의 과정.

chain = final_prompt | llm # LECL 활용. 

# # 체인 실행 예시 1 (질문 분해)
response = chain.invoke({"input": \
                         "'나는 서울 사는 26살이고 현재 미취업 상태야. 국취제 1 유형을 듣고 있고 교육 수당은 총 100만원을 받고 있는데 내가 받을 수 있는 정책좀 그리고 링크도'"})

# # 결과 출력
print(response.content)
# print(response)
# # 예상 출력:
# # ● 최근 5년간 종로구 인구 현황 알려줘
# # ● 최근 5년간 종로구 인구 추이 알려줘


● 내가 받을 수 있는 정책은 무엇인가요?
● 정책의 링크도 제공해줄 수 있나요?● 내가 받을 수 있는 정책은 무엇인가요?
● 정책의 링크도 제공해줄 수 있나요?


# 화행분류 (Speech-Act Classification) 
챗봇의 다음 행동을 결정하는 '라우터(Router' 또는 '디스패처(Dispatcher)'
사용자가 원하는 작업의 종류를 먼저 파악하고, 그에 맞는 전문적인 도구(체인)에게 일을 넘겨줌.


In [28]:
# 1. 시스템 프롬프트 정의
system_prompt = """
다음은 질문입니다.
당신은 질문에 대해 분류를 합니다.
해당 질문이 테이블의 목록을 조회하는 질문인지, 테이블 내 값을 조회하는 질문인지, 보고서를 작성해달라는 질문인지, 그 외 질문인지를 판단하십시오.
답은 "목록 조회", "테이블값 조회", "보고서 작성", "그 외" 4개로 한정합니다.
어떤 유형의 질문인지 파악할 수 없는 경우 "그 외"로 분류합니다.
음식점 이름 등 상호명을 직접 입력한 경우 "그 외"로 분류합니다.
서울이 아닌 다른 지역의 데이터를 묻는 경우 "그 외"로 분류합니다.
공무원의 평균 연령을 묻는 경우 "그 외"로 분류합니다.
"""

In [29]:
# 2. few-shot 데이터 예제 정의

examples = [
    {
        "input": "전기차 관련 통계 있어?",
        "output": "목록 조회"
    },
    {
        "input": "서울시 마포구에 있는 전기차 충전소의 주소를 알려줘",
        "output": "테이블값 조회"
    },
    {
        "input": "인구밀도와 전세가의 관계 알려줘",
        "output": "보고서 작성"
    },
    {
        "input": "땀땀 위치 알려줘.",
        "output": "그 외"
    },
    {
        "input": "서울시 자치구별 소득 평균에 대해서 표를 만들어줘.",
        "output": "테이블값 조회"
    },
    {
        "input": "경기도 인구 알려줘",
        "output": "그 외"
    },
    {
        "input": "자치구별 월 평균 소득 비교해줘",
        "output": "보고서 작성"
    }
]

In [34]:
# 3. 템플릿 정의

example_prompt = ChatPromptTemplate.from_messages(
    [ ("human", "{input}"),
       ("ai", "{output}"),
    ]
)

# 4. few-shot 프롬프트 템플릿 생성
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

In [35]:
# 5. 최종 프롬프트 템플릿 조립
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

In [45]:
from langchain_core.output_parsers import StrOutputParser

# 6. 체인 구성 및 실행
# output_parser = StrOutputParser()

# chain = final_prompt | llm | output_parser
chain = final_prompt | llm

In [56]:
# 체인 실행 예시
user_query = "최근 3년 면적 대비 부동산 실거래가 가격 분석해줘"
action = chain.invoke({"input": user_query})
print(f"질문: {user_query}")
print(f"분류된 행동: {action}") # 예상 출력: 보고서 작성

user_query_2 = "서울시 도로시설물에 관한 자료 있어?"
action_2 = chain.invoke({"input": user_query_2})
print(f"\n질문: {user_query_2}")
print(f"분류된 행동: {action_2}") # 예상 출력: 목록 조회

질문: 최근 3년 면적 대비 부동산 실거래가 가격 분석해줘
분류된 행동: 보고서 작성

질문: 서울시 도로시설물에 관한 자료 있어?
분류된 행동: 목록 조회


# 질의 정제

In [57]:
# 일반적인 질의 정제

system_prompt = """
당신은 사용자의 질문을 데이터베이스 검색에 최적화된 형태로 정제하는 AI입니다.
다음 규칙에 따라 주어진 질문을 가장 명확하고 간결한 검색어로 변환하십시오.

1. 질문의 핵심 의도와 키워드는 반드시 유지합니다.
2. 오타나 문법적 오류를 수정합니다.
3. "알려줘", "궁금해요", "혹시" 등 불필요한 미사여구나 구어적 표현을 제거합니다.
4. 검색에 유리하도록 핵심 단어 중심으로 질문을 재구성합니다. 하지만 단어의 의미를 창작하거나 바꾸지 마십시오.
"""
examples = [
    {
        "input": "음... 혹시 서울에 있는 공원 중에 강아지랑 같이 갈 수 있는 곳 좀 알려줄 수 있어?",
        "output": "서울시 반려견 동반 가능 공원 목록"
    },
    {
        "input": "서울 청년 월세 지원 조건이 어떻게 되나요?",
        "output": "서울특별시 청년월세지원 신청자격"
    },
    {
        "input": "서울시 마포구에 있는 전키차 충전소 주소",
        "output": "서울시 마포구 전기차 충전소 주소"
    },
    {
        "input": "따릉이 대여소 위치",
        "output": "따릉이 대여소 위치"
    },
     {
        "input": "최근 5년간 종로구 인구 추이랑 현황 알려줘",
        "output": "최근 5년간 종로구 인구 현황 및 추이"
    }
]


In [59]:

# --- 3. 개별 예제를 위한 템플릿 정의 ---
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# --- 4. Few-shot 프롬프트 템플릿 생성 ---
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# --- 5. 최종 프롬프트 템플릿 조립 ---
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# --- 6. 체인 구성 및 실행 ---
output_parser = StrOutputParser()
refinement_chain = final_prompt | llm | output_parser

# # 체인 실행 예시
original_query = "작년에 서울에서 폐업한 식당들 리스트가 궁금해요"
refined_query = refinement_chain.invoke({"input": original_query})
print(f"원본 질문: {original_query}")
print(f"정제된 질문: {refined_query}")

2022년 서울 폐업 식당 리스트원본 질문: 작년에 서울에서 폐업한 식당들 리스트가 궁금해요
정제된 질문: 2022년 서울 폐업 식당 리스트


In [66]:
# 강사님 엑셀 기반 질의 정제

# 1. 시스템 프롬프트 설정
system_prompt = """
당신은 주어진 질문을 데이터 열 예시로 변경하면서 주어진 데이터셋으로 질문에 답변할 수 있는지 판단하는 '서울 열린데이터 광장'챗봇의 과정 중 하나입니다.
다음은 질문과 데이터 이름, 데이터 열 예시입니다.
주어진 질문을 제공된 데이터 이름과 데이터 열로 답변이 가능한지 여부를 가능 여부에 "O", "X"로 답변하시오.
답변이 가능하면 주어진 질문을 데이터 열 예시로 변경해서 변환 질문에 반환하세오.
답변이 불가능하면 변환 질문에 "질문에 답변할 수 없는 데이터 셋 입니다."를 반환하세요.
답변은 "- 가능 여부:\\n{{}}\\n- 변환 질문:\\n{{}}"형식입니다.

주의사항:
본 질문의 맥락을 스스로 판단해서 질문 범위를 한정하지 마십시오.
만약 변환이 이상해진다면, 원래의 질문을 그대로 반환하십시오.
데이터 열 이름을 추가하는 것이 아니라 대체하는 것 입니다.
"""

# 2. few-shot 예제 정의
# 입력 변수가 여러 개 이므로, 그에 맞게 템플릿 수정
example_input_variables = ["question", "dataset_name", "columns"] 
examples = [
    {
        "question": "종로구에 있는 모든 도서관이 보유한 도서 개수의 총합은 얼마나 돼?",
        "dataset_name": "서울시 공공도서관 현황",
        "columns": "도서관명, 서비스분류코드, 서비스분류, 자치구코드, 자치구명, 연령구분코드, 연령구분, 시설분류코드, 도서수, X좌표값, Y좌표값, 우편번호, 기본주소, 상세주소, 사용료무료여부, 사용료, 안내URL, 등록일시, 수정일시",
        "answer": "- 가능 여부:\nO\n- 변환 질문:\n종로구에 있는 모든 도서관이 보유한 도서수의 총합은 얼마나 돼?"
    },
    {
        "question": "강남구에 급속충전기수가 3대 이상인 곳의 이름과 위치를 알려줘",
        "dataset_name": "서울시 급속충전기 현황",
        "columns": "충전소명, 충전소위치상세, 설치시도명, 휴점일, 이용가능시작시각, 이용가능종료시각, 완속충전가능여부, 급속충전가능여부, 급속충전타입구분, 완속충전기대수, 급속충전기대수, 주차료부과여부, 소재지도로명주소, 소재지지번주소, 관리업체명, 관리업체전화번호, 위도, 경도, 데이터기준일자",
        "answer": "- 가능 여부:\nO\n- 변환 질문:\n강남구에 급속충전기대수가 3대 이상인 곳의 충전소명과 충전소위치상세를 알려줘"
    },
    {
        "question": "성북구 주차장 알려줘",
        "dataset_name": "한강공원 주차장 정보",
        "columns": "지구별, 주차장별, 위치정보(위도), 위치정보(경도), 주소, 면수, 장애면수, 정기면수, 기본시간, 기본요금, 간격시간, 간격요금, 전일요금",
        "answer": "- 가능 여부:\nX\n- 변환 질문:\n질문에 답변할 수 없는 데이터 셋 입니다."
    },
    {
        "question": "강동구 세탁업체 개수 알려줘",
        "dataset_name": "서울시 금천구 세탁업 인허가 정보",
        "columns": "인허가일자, 영업상태명, 상세영업상태명, 폐업일자, 주소, 사업장명, 업태구분명",
        "answer": "- 가능 여부:\nX\n- 변환 질문:\n질문에 답변할 수 없는 데이터 셋 입니다."
    }
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "질문:\n{question}\n데이터셋 이름:\n{dataset_name}\n데이터 열 예시:\n{columns}"),
        ("ai", "{answer}"),
    ]
)


# 3. few-shot 템플릿 생성
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    input_variables=example_input_variables,
)

# --- 4. 최종 프롬프트 템플릿 조립 ---
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        few_shot_prompt,
        ("human", "질문:\n{question}\n데이터셋 이름:\n{dataset_name}\n데이터 열 예시:\n{columns}"),
    ]
)

In [68]:
# --- 5. 체인 구성 및 실행 ---
output_parser = StrOutputParser()
grounding_chain = final_prompt | llm | output_parser

# # 체인 실행 예시
user_question = "따릉이 고장원인들의 비율 알려줘"
selected_dataset = "서울시 공공자전거 고장신고 내역"
dataset_columns = "자전거번호, 등록일(YYYYMMDD), 고장구분"

response = grounding_chain.invoke({
    "question": user_question,
    "dataset_name": selected_dataset,
    "columns": dataset_columns
})

- 가능 여부:
O
- 변환 질문:
따릉이 고장구분의 비율 알려줘